🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys.

# Named Entity Recognition with Haystack Pipelines

## Overview
This notebook demonstrates how to build an end-to-end **Named Entity Recognition (NER)** pipeline using Haystack 2.0. We'll create a system that searches the web for articles, extracts content, and automatically identifies and categorizes named entities (people, organizations, locations, and miscellaneous entities).

## What You'll Learn
By the end of this notebook, you will understand:
1. How to build modular Haystack pipelines with multiple components
2. How to integrate web search capabilities into NLP workflows
3. How to extract and process HTML content from web pages
4. How to use pre-trained NER models with Haystack
5. How to create custom Haystack components for specialized data processing
6. How to structure extracted entities for downstream analysis

## Use Cases
This pipeline pattern is useful for:
- **Media Monitoring**: Track mentions of people, organizations, and places in news articles
- **Research**: Gather and categorize information about specific topics or entities
- **Content Analysis**: Automatically tag and organize web content by entities
- **Knowledge Base Creation**: Extract structured information from unstructured web data

## Pipeline Architecture
Our pipeline follows this flow:
```
Web Search → Fetch URLs → Convert HTML → Clean Text → Extract Entities → Structure Results
```

Each component performs a specific task, and they're connected to create a complete workflow.

In [1]:
from haystack import Pipeline
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.websearch import SearchApiWebSearch
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.writers import DocumentWriter
from haystack.utils import Secret
from haystack import Pipeline
from haystack.components.extractors import NamedEntityExtractor
from haystack import component, Document
from typing import Any, Dict, List, Union

from dotenv import load_dotenv
import os
from pathlib import Path

# Load .env from the root of ch8 directory
root_dir = Path(__file__).parent.parent if "__file__" in globals() else Path.cwd().parent
load_dotenv(root_dir / ".env")

open_ai_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPERDEV_API_KEY")


## Step 1: Import Required Libraries

We'll import all necessary components from Haystack and supporting libraries:

**Haystack Components:**
- **`Pipeline`**: The core container that orchestrates all components
- **`SearchApiWebSearch`**: Performs web searches using SearchAPI
- **`LinkContentFetcher`**: Downloads content from URLs
- **`HTMLToDocument`**: Converts HTML to Haystack Document objects
- **`DocumentCleaner`**: Preprocesses text by removing unwanted content
- **`NamedEntityExtractor`**: Identifies named entities using ML models
- **`Secret`**: Securely manages API keys from environment variables

**Supporting Libraries:**
- **`dotenv`**: Loads environment variables from .env file
- **`typing`**: Provides type hints for better code documentation

## Step 2: Define a Custom Haystack Component

### What is a Custom Component?
Haystack allows you to create custom components to handle specialized processing tasks. A custom component must:
1. Be decorated with `@component`
2. Define input/output types using `@component.output_types()`
3. Implement a `run()` method that processes data

### Why Create NERPopulator?
The built-in `NamedEntityExtractor` identifies entities, but we need to:
- Filter entities by confidence score (only keep high-quality predictions)
- Remove duplicates within each entity type
- Organize entities into a structured format (LOC, PER, ORG, MISC)
- Preserve source URLs for traceability

### Component Architecture
Our `NERPopulator` component:
- **Input**: List of documents with `named_entities` in metadata
- **Processing**: Filters, deduplicates, and categorizes entities
- **Output**: Structured documents ready for analysis or storage

In [2]:
@component
class NERPopulator():
    """This function extracts named entities from a list of
    documents and returns the result in a structured format, and saves to CSV.

    Args:
        documents (list): List of Haystack Document objects
        output_file (str): Optional path for the CSV output file. If not provided, auto-generates filename.

    Returns:
        extracted_data (list): A list of dictionaries containing the extracted entities, 
        csv_path (str): Path to the saved CSV file
    """
    
    def __init__(self, output_dir: str = "./ner_results"):
        """
        Initialize NERPopulator with output directory.
        
        Args:
            output_dir: Directory where CSV files will be saved
        """
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)
    
    @component.output_types(documents=List[Document], csv_path=str, summary=str)
    def run(self, sources: List[Document], output_file: str = None) -> Dict[str, Any]:
        import pandas as pd
        from datetime import datetime
        
        extracted_data = []

        for document in sources:
            content = document.content
            doc_id = document.id
            named_entities = document.meta.get('named_entities', [])
            url = document.meta.get('url', 'N/A')  # Default to 'N/A' if URL is not available

            # Sets to store unique entities by type
            entities_by_type = {
                "LOC": set(),
                "PER": set(),
                "ORG": set(),
                "MISC": set()
            }
            
            # Loop through the entities and filter by score and type
            for entity in named_entities:
                if float(entity.score) < 0.8:
                    continue
                
                word = content[entity.start:entity.end]
                if entity.entity in entities_by_type:
                    entities_by_type[entity.entity].add(word)  # Use set to ensure uniqueness
            
            # Prepare the meta field with comma-separated values
            meta = {
                "LOC": ",".join(entities_by_type["LOC"]),
                "PER": ",".join(entities_by_type["PER"]),
                "ORG": ",".join(entities_by_type["ORG"]),
                "MISC": ",".join(entities_by_type["MISC"]),
                "url": url
            }
            
            # Append the result for this document
            extracted_data.append({
                'document_id': doc_id,
                'content': content,
                'meta': meta
            })
        
        # Generate CSV filename if not provided
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"ner_results_{timestamp}.csv"
        
        csv_path = os.path.join(self.output_dir, output_file)
        
        # Create DataFrame and expand meta column
        df = pd.DataFrame(extracted_data)
        meta_df = pd.json_normalize(df['meta'])
        result_df = pd.concat([df.drop('meta', axis=1), meta_df], axis=1)
        
        # Save to CSV
        result_df.to_csv(csv_path, index=False)
        
        # Create summary
        total_docs = len(extracted_data)
        all_entities = {
            "LOC": set(),
            "PER": set(),
            "ORG": set(),
            "MISC": set()
        }
        
        for doc in extracted_data:
            for entity_type in all_entities:
                entities = doc['meta'][entity_type]
                if entities:
                    all_entities[entity_type].update(entities.split(','))
        
        summary = f"Processed {total_docs} documents.\n"
        summary += f"Found {len(all_entities['PER'])} unique people, "
        summary += f"{len(all_entities['ORG'])} organizations, "
        summary += f"{len(all_entities['LOC'])} locations, "
        summary += f"and {len(all_entities['MISC'])} miscellaneous entities.\n"
        summary += f"Results saved to: {csv_path}"

        return {
            "documents": extracted_data,
            "csv_path": csv_path,
            "summary": summary
        }


### Understanding the NERPopulator Implementation

**Key Features:**

1. **Confidence Filtering**: `if float(entity.score) < 0.8: continue`
   - Only entities with >80% confidence are kept
   - Reduces false positives and improves data quality

2. **Entity Categories (Standard NER labels)**:
   - **LOC**: Locations (cities, countries, landmarks)
   - **PER**: Persons (names of people)
   - **ORG**: Organizations (companies, institutions)
   - **MISC**: Miscellaneous entities (events, products, etc.)

3. **Deduplication with Sets**: `entities_by_type[entity.entity].add(word)`
   - Using Python sets automatically removes duplicate entities
   - Important when the same entity appears multiple times in text

4. **Metadata Preservation**:
   - Original URL is maintained for source tracking
   - Entities are stored as comma-separated strings for easy export

**Output Format:**
Each processed document contains:
- `document_id`: Unique identifier
- `content`: Full text content
- `meta`: Dictionary with entity categories and source URL

## Step 3: Build the Complete Haystack Pipeline

### Pipeline Components Explained

In this step, we'll initialize each component and connect them into a working pipeline.

**Component Configuration:**

1. **SearchApiWebSearch**
   - `top_k=5`: Returns the top 5 search results
   - `api_key`: Loaded securely from environment variables
   - `allowed_domains`: Restricts results to Britannica.com for quality content

2. **LinkContentFetcher**
   - `retry_attempts=3`: Retries failed downloads up to 3 times
   - `timeout=10`: Waits maximum 10 seconds per request

3. **HTMLToDocument**
   - Converts raw HTML into structured Haystack Document objects

4. **DocumentCleaner**
   - `remove_empty_lines=True`: Removes blank lines
   - `remove_extra_whitespaces=True`: Normalizes spacing
   - `remove_substrings=['\n-']`: Removes specific unwanted patterns

5. **NamedEntityExtractor**
   - `backend="hugging_face"`: Uses HuggingFace models
   - `model="dslim/bert-base-NER"`: Pre-trained BERT model fine-tuned for NER
   - `warm_up()`: Loads the model into memory for faster inference

6. **NERPopulator**
   - Our custom component for structuring the results

In [3]:

# Initialize pipeline
pipeline = Pipeline()
web_search = SearchApiWebSearch(top_k=5,
                                api_key=Secret.from_env_var("SEARCH_API_KEY"),
                                allowed_domains=["https://www.britannica.com/"])
link_content = LinkContentFetcher(retry_attempts=3,
                                  timeout=10)
html_to_doc = HTMLToDocument()
document_cleaner = DocumentCleaner(
                                remove_empty_lines=True,
                                remove_extra_whitespaces=True,
                                remove_repeated_substrings=False,
                                remove_substrings=['\n-']
                            )
extractor = NamedEntityExtractor(backend="hugging_face", model="dslim/bert-base-NER")
extractor.warm_up()

ner_component = NERPopulator(output_dir="./ner_results")

# Add components
pipeline.add_component(name='search', instance=web_search)
pipeline.add_component(name ='fetcher' , instance= link_content)
pipeline.add_component(name='htmldocument', instance=html_to_doc)
pipeline.add_component(name='cleaner', instance=document_cleaner)
pipeline.add_component(name='extractor', instance=extractor)
pipeline.add_component(name='ner', instance=ner_component)

# Connect components to one another
pipeline.connect("search.links", "fetcher.urls")
pipeline.connect("fetcher", "htmldocument")
pipeline.connect("htmldocument", "cleaner")
pipeline.connect("cleaner", "extractor")
pipeline.connect("extractor", "ner")


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps


🚅 Components
  - search: SearchApiWebSearch
  - fetcher: LinkContentFetcher
  - htmldocument: HTMLToDocument
  - cleaner: DocumentCleaner
  - extractor: NamedEntityExtractor
  - ner: NERPopulator
🛤️ Connections
  - search.links -> fetcher.urls (list[str])
  - fetcher.streams -> htmldocument.sources (list[ByteStream])
  - htmldocument.documents -> cleaner.documents (list[Document])
  - cleaner.documents -> extractor.documents (list[Document])
  - extractor.documents -> ner.sources (list[Document])

### Understanding Pipeline Connections

**The Data Flow:**

```
1. search (SearchApiWebSearch)
   ↓ outputs: links
2. fetcher (LinkContentFetcher)
   ↓ outputs: streams (raw HTML)
3. htmldocument (HTMLToDocument)
   ↓ outputs: documents
4. cleaner (DocumentCleaner)
   ↓ outputs: documents (cleaned)
5. extractor (NamedEntityExtractor)
   ↓ outputs: documents (with named_entities in meta)
6. ner (NERPopulator)
   ↓ outputs: documents (structured)
```

**Key Concepts:**

- **Component Naming**: Each component gets a unique name (e.g., 'search', 'fetcher')
- **Output Routing**: We specify which output connects to which input
  - Example: `"search.links"` → `"fetcher.urls"`
- **Automatic Type Matching**: Haystack validates that outputs match expected inputs
- **Sequential Processing**: Each component processes data from the previous one

**Why This Order Matters:**
1. Can't extract entities before fetching content
2. Can't clean text before converting HTML
3. Can't structure results before extracting entities

This is a **linear pipeline** - each step depends on the previous one.

## Step 4: Run the Pipeline

### How Pipeline Execution Works

When you call `pipeline.run()`, Haystack:
1. Passes your input to the first component ('search')
2. Each component processes data and passes results to the next
3. Returns the final output from the last component

**Input Format:**
```python
data={"search": {"query": query}}
```
- Dictionary keys match component names
- Nested dictionary contains the component's input parameters
- For 'search', we only need to provide the 'query' parameter

**What Happens During Execution:**
1. SearchAPI finds 5 relevant articles about "Elon Musk"
2. LinkContentFetcher downloads HTML from each URL
3. HTMLToDocument converts HTML to text
4. DocumentCleaner removes noise and formatting
5. NamedEntityExtractor identifies people, places, organizations
6. NERPopulator structures the entities into our custom format

**Expected Processing Time:**
- Web search: ~1-2 seconds
- Content fetching: ~3-10 seconds (depends on website response)
- NER extraction: ~5-10 seconds (model inference)
- Total: ~10-20 seconds for 5 articles

In [4]:
query = "Elon Musk"
output = pipeline.run(data={"search":{"query":query}})

### Understanding the Output Structure

The `output` variable contains results from all pipeline components. Structure:
```python
{
    'search': {...},      # Search results
    'fetcher': {...},     # Downloaded content
    'htmldocument': {...},# Converted documents
    'cleaner': {...},     # Cleaned documents
    'extractor': {...},   # Documents with raw entities
    'ner': {              # Our final structured output
        'documents': [...]
    }
}
```

We're interested in the final output from our custom 'ner' component.

In [5]:
extracted_documents = output['ner']['documents']

## Step 5: Extract and Examine Results

Now we'll extract just the final processed documents from our pipeline output.

**What's in extracted_documents?**
Each document is a dictionary containing:
- `document_id`: Unique identifier
- `content`: Full article text
- `meta`: Dictionary with:
  - `LOC`: Comma-separated locations (e.g., "California,Texas,Mars")
  - `PER`: Comma-separated person names (e.g., "Elon Musk,Jeff Bezos")
  - `ORG`: Comma-separated organizations (e.g., "Tesla,SpaceX,NASA")
  - `MISC`: Miscellaneous entities (e.g., "Cybertruck,Model 3")
  - `url`: Source URL for the article

## Step 6: Convert to DataFrame for Analysis

### Why Use a DataFrame?

Converting our results to a pandas DataFrame provides:
- **Tabular View**: Easy-to-read table format
- **Data Analysis**: Use pandas methods for filtering, grouping, aggregating
- **Export Options**: Save to CSV, Excel, or databases
- **Visualization**: Create charts and graphs

### What You'll See

The DataFrame will have columns:
- `document_id`: Unique ID for each article
- `content`: Full article text (may be long)
- `meta`: Nested dictionary with entity categories and URL

**Tip**: You can expand the meta column to see all extracted entities for each document.

In [6]:
import pandas as pd
df = pd.DataFrame(extracted_documents)

df

,document_id,content,meta
0,b673a37853a56857b7ca61fa0597deb527c8b1aff16e1f...,Elon Musk\nWhen was Elon Musk born?\nElon Musk...,"{'LOC': 'South,Pretoria,Philadelphia,States,Ki..."
1,029037a6d9de96ba5d6150dfd119c79ef8afb66895aef8...,Elon Musk went to Queen’s University in Kingst...,"{'LOC': 'Kingston,Canada,Ontario,Philadelphia'..."
2,f85e239cb34a29728ea742ababe1795dec329132e6bf2b...,"Tesla, Inc. formerly (2003–17): Tesla Motors D...","{'LOC': 'Austin,Texas', 'PER': 'Nikola Tesla,M..."
3,3808990b1c9929bc0c40cca51a77685593e5354bbd2bd4...,"Tesla under Musk: New models, battery technolo...","{'LOC': 'Mexico,Buffalo,Europe,United States,S..."
4,c232b1eab7bc6b14cddbd7fc5c1f636c91a92e92064731...,Zip2 Date: 1995 - 1999 Ticker: HPQ Share price...,"{'LOC': '', 'PER': 'Enrique J,Kim,Gregory Ko,o..."


### Exploring the DataFrame

**Useful operations you can try:**

```python
# See all columns
df.columns

# View just the metadata
df['meta']

# Access specific entity types for first document
df['meta'][0]['PER']  # People mentioned
df['meta'][0]['ORG']  # Organizations mentioned
df['meta'][0]['LOC']  # Locations mentioned

# Count documents
len(df)

# Save to CSV
df.to_csv('ner_results.csv', index=False)
```

## Step 7: Creating a SuperComponent

### What is a SuperComponent?

A SuperComponent wraps an entire pipeline into a single reusable component with a simplified interface. Benefits:

1. **Simplified Interface**: Map multiple pipeline inputs to a single input parameter
2. **Encapsulation**: Hide complex internal pipeline structure
3. **Reusability**: Use the pipeline as a component in larger systems
4. **Tool Creation**: Easily convert to a tool for agent-based systems

### Our NER SuperComponent

We'll create a SuperComponent that:
- Takes a natural language query and optional website domain
- Internally distributes the query to the search component
- Returns structured entity data and CSV file path

In [7]:
from haystack import SuperComponent

# Create a SuperComponent with simplified input/output mapping
ner_super_component = SuperComponent(
    pipeline=pipeline,
    input_mapping={
        "query": ["search.query"],
        "allowed_domains": ["search.query"]
    },
    output_mapping={
        "ner.documents": "documents",
        "ner.csv_path": "csv_path",
        "ner.summary": "summary"
    }
)

print("NER Pipeline has been successfully wrapped into a SuperComponent.")

NER Pipeline has been successfully wrapped into a SuperComponent.


## Step 8: Test the SuperComponent

Let's test our SuperComponent with a simple query to ensure it works correctly before creating the agent.

In [8]:
# Test the SuperComponent directly
test_result = ner_super_component.run(
    query="Marie Curie"
)

# Display the summary
print(test_result['summary'])
print(f"\nCSV file saved at: {test_result['csv_path']}")

Processed 5 documents.
Found 25 unique people, 17 organizations, 18 locations, and 13 miscellaneous entities.
Results saved to: ./ner_results/ner_results_20251124_222007.csv

CSV file saved at: ./ner_results/ner_results_20251124_222007.csv


## Step 9: Creating a Tool from the SuperComponent

Now we'll wrap our SuperComponent into a ComponentTool that can be used by an AI agent. This allows the agent to:
- Extract entities from any website based on natural language queries
- Process complex user requests like "Find all people mentioned on Wikipedia articles about Einstein"
- Automatically save results to CSV files

### Tool Configuration

The tool needs:
1. **Name**: A simple identifier for the agent to reference
2. **Description**: Clear explanation of what the tool does (crucial for agent decision-making)
3. **Component**: Our NER SuperComponent

In [9]:
from haystack.tools.component_tool import ComponentTool

# Create a tool from the SuperComponent
ner_tool_name = "entity_extraction_tool"

ner_tool_description = (
    "Use this tool to extract named entities (people, organizations, locations, and miscellaneous entities) "
    "from web articles. Provide a query string describing what to search for (e.g., 'Albert Einstein', 'Tesla company') "
    "and optionally specify allowed_domains as a list of website URLs to restrict the search. "
    "The tool will search the web, fetch articles, and extract all named entities, saving results to a CSV file. "
    "Returns a summary of extracted entities and the path to the CSV file."
)

ner_tool = ComponentTool(
    name=ner_tool_name,
    component=ner_super_component,
    description=ner_tool_description,
)

print("NER SuperComponent has been successfully wrapped into a Tool for agent use.")

NER SuperComponent has been successfully wrapped into a Tool for agent use.


## Step 10: Building an Agent with the NER Tool

### Agent Architecture

An agent is an intelligent system that can:
1. Understand natural language requests
2. Decide which tools to use
3. Execute tools with appropriate parameters
4. Synthesize results into coherent responses

### Our NER Agent Configuration

We'll create an agent that:
- Uses GPT-4 for reasoning and decision-making
- Has access to our entity extraction tool
- Can parse user requests like "Extract entities about Marie Curie from Britannica"
- Handles the extraction and returns both the summary and CSV file location

### System Prompt

The system prompt guides the agent's behavior and teaches it how to use the tools effectively.

In [ ]:
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage


# # Initialize the agent's LLM (must support tool calling)
# agent_llm = OpenAIChatGenerator(
#     api_key=Secret.from_env_var("OPENAI_API_KEY"),
#     model="gpt-4o-mini"
# )

# Or use Ollama models (uncomment to use):
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

agent_llm = OllamaChatGenerator(
    model="mistral-nemo:12b",
    generation_kwargs={
        "num_predict": 100,
        "temperature": 0.9,
    }
)

# Define the system prompt to guide the agent
system_prompt = """
You are a helpful Named Entity Recognition assistant. Your goal is to help users extract 
named entities (people, organizations, locations, and miscellaneous entities) from web articles.

You have access to the entity_extraction_tool which can:
- Search the web for articles based on a query
- Extract named entities from the articles found
- Save results to a CSV file

When a user asks you to extract entities:
1. Identify the search query from their request
2. Call the entity_extraction_tool with the appropriate parameters
3. Present the results including:
   - Summary of entities found
   - Path to the CSV file
   - Key entities discovered (people, organizations, locations)

Be conversational and helpful. Parse natural language requests carefully.
"""

# Create the agent with the NER tool
tools = [ner_tool]

ner_agent = Agent(
    chat_generator=agent_llm,
    tools=tools,
    system_prompt=system_prompt
)

print("NER Agent successfully created and ready to process queries!")

NER Agent successfully created and ready to process queries!


## Step 11: Using the Agent with Natural Language Queries

Now let's put our agent to work! We'll send natural language queries that specify:
1. What topic to search for
2. Optionally, which website to search

The agent will:
- Parse the request
- Call the entity extraction tool with appropriate parameters
- Return a summary and CSV file path

### Example Query Format

Natural language examples:
- "Extract entities about Albert Einstein from Britannica"
- "Find all people and organizations mentioned in articles about Marie Curie"
- "Search for entities related to Tesla on Britannica"

In [12]:
# Example 1: Simple query with default website
query1 = "Extract all named entities about Albert Einstein and Elon Musk"

print(f"User Query: {query1}\n")
print("Agent is processing your request...\n")

agent_result1 = ner_agent.run(messages=[ChatMessage.from_user(query1)])

User Query: Extract all named entities about Albert Einstein and Elon Musk

Agent is processing your request...



### Display Agent Response

Let's create a helper function to nicely display the agent's response and show the workflow.

In [13]:
from IPython.display import Markdown, display

def display_agent_result(agent_result):
    """
    Display the agent's reasoning process and final response in a formatted way.
    
    Args:
        agent_result: The result dictionary from the agent's run
    """
    # Display user query
    user_query = agent_result['messages'][1]._content[0].text
    display(Markdown(f"### User Query\n*{user_query}*\n"))
    
    # Display tool calls
    display(Markdown("### Agent Actions"))
    for msg in agent_result['messages']:
        if msg._role.value == 'assistant' and hasattr(msg._content[0], 'tool_name'):
            for tool_call in msg._content:
                display(Markdown(f"**Tool Used:** `{tool_call.tool_name}`"))
                display(Markdown(f"**Parameters:**"))
                for key, value in tool_call.arguments.items():
                    display(Markdown(f"- {key}: `{value}`"))
                display(Markdown(""))
    
    # Display final response
    display(Markdown("### Agent Response"))
    final_response = agent_result['last_message']._content[0].text
    display(Markdown(final_response))

# Display the result
display_agent_result(agent_result1)

### User Query
*Extract all named entities about Albert Einstein and Elon Musk*


### Agent Actions

**Tool Used:** `entity_extraction_tool`

**Parameters:**

- allowed_domains: `https://www.britannica.com https://www.wikipedia.org`

- query: `'Albert Einstein' AND 'Elon Musk'`

### Agent Response

Here's a summary of the extracted information from both texts:

**Text 1:**
- **People:** None mentioned
- **Organizations:** Britannica
- **Locations:** United States, Atlantic Ocean, Gulf of Mexico, Pacific Ocean
- **Miscellaneous:** Progressive era, American Revolution

**Text 2:**
- **People:** Donald Trump
- **Organizations:** U.S. (mentioned twice)
- **Locations:** United States, Washington D.C., China, Russia

### Example 2: Query with Custom Website

Let's try another query where we specify a different website domain.

In [ ]:
# Example 2: Query specifying a different topic
query2 = "Find entities about Nikola Tesla from Britannica and save them to a CSV file"

print(f"User Query: {query2}\n")
print("Agent is processing your request...\n")

agent_result2 = ner_agent.run(messages=[ChatMessage.from_user(query2)])
display_agent_result(agent_result2)

### Viewing the Saved CSV Files

You can load and inspect the CSV files that were generated:

In [ ]:
# List all CSV files in the results directory
import glob

csv_files = glob.glob("./ner_results/*.csv")
print(f"Found {len(csv_files)} CSV files:\n")
for file in csv_files:
    print(f"- {file}")

# Load and display the most recent CSV
if csv_files:
    latest_csv = max(csv_files, key=os.path.getctime)
    print(f"\nLoading most recent file: {latest_csv}\n")
    
    df_result = pd.read_csv(latest_csv)
    display(Markdown(f"### Results from {latest_csv}"))
    display(df_result.head())

## Summary and Key Takeaways

### What We've Accomplished

1. **Enhanced NER Pipeline**: Modified the NERPopulator component to automatically save results to CSV files
2. **Created a SuperComponent**: Wrapped the entire pipeline with a simplified interface
3. **Built a Tool**: Converted the SuperComponent into a tool that agents can use
4. **Implemented an Agent**: Created an intelligent agent that can process natural language queries

### Key Features

- ✅ **Natural Language Interface**: Users can ask questions in plain English
- ✅ **Automatic CSV Generation**: Results are automatically saved with timestamps
- ✅ **Entity Categorization**: People, organizations, locations, and miscellaneous entities are separated
- ✅ **Website Flexibility**: Can search specific domains or use defaults
- ✅ **Summary Reports**: Get quick insights before diving into the CSV

### Use Cases

This system is ideal for:
- **Research**: Quickly extract entities from multiple articles about a topic
- **Content Analysis**: Identify key people, places, and organizations in web content
- **Knowledge Base Building**: Systematically extract structured data from unstructured text
- **Media Monitoring**: Track entity mentions across different sources

### Next Steps

You can extend this system by:
- Adding more tools (e.g., sentiment analysis, summarization)
- Implementing custom entity types
- Integrating with databases for persistent storage
- Creating visualizations of entity relationships
- Adding entity disambiguation and linking

## Transition to SuperComponent and Agent Architecture

In the following sections, we'll take our NER pipeline to the next level by:

1. **Creating a SuperComponent**: Wrapping our pipeline for easier reuse
2. **Building a Tool**: Making the pipeline accessible to AI agents
3. **Implementing an Agent**: Creating an intelligent assistant that can process natural language queries

This architecture allows users to simply ask questions like "Extract entities about Einstein from Britannica" instead of manually configuring pipeline parameters.